### Find Unusual Patterns in Hourly Google Search Traffic
The goal here is to find out if there is a link between financial events for MercadoLibre and Google search traffic. 
We will dive into May 2020, when MercadoLibre released a quarterly financial report

In [150]:
import pandas as pd
import holoviews as hv
import hvplot.pandas
import seaborn as sns
import numpy as np
from datetime import datetime
from pytz import timezone

In [170]:
# Read in the search trends data
meli_search_df = pd.read_csv('resources/google_hourly_search_trends.csv', infer_datetime_format=True, parse_dates=True)

# Slice the frame to just the month of May 2020. We first need to convert our Date column to Datetime
meli_search_df['Date'] = pd.to_datetime(meli_search_df['Date'], infer_datetime_format=True, utc=True)

# Convert the Date Column to US/Eastern time
#meli_search_df['Date'] = meli_search_df['Date'].dt.tz_convert('US/Eastern')

# Set the index to our Datetime column for simple lookup / plot statements
meli_search_df = meli_search_df.set_index('Date')

In [171]:
meli_search_df.tail()

,Search Trends
Date,
2020-09-07 20:00:00+00:00,71
2020-09-07 21:00:00+00:00,83
2020-09-07 22:00:00+00:00,96
2020-09-07 23:00:00+00:00,97
2020-09-08 00:00:00+00:00,96


In [172]:
# Because the end of the dataset coincides with the beginning of the month, 
# We will trim data from Sept. 2020 from the data set so that our measures of centrality are more representative

meli_search_df = meli_search_df.loc[:'2020-08-31']


## Zoom in on May 2020, month of Quarterly Financial Report

In [173]:

fig_march = meli_search_df.loc['2020-05'].hvplot(title= 'May 2020 Search Trend Data')
fig_march

:Curve   [Date]   (Search Trends)

### Interesting Spikes on May 5th 
The chart appears to be a stochastic process oscillating between ~10 and ~100 with reliable cyclicality. However, the data for **May 5th** seems elevated relative to the rest of the month. Let's see if the entire month was anomalous.

### Compare May 2020 to Monthly Median


In [174]:
#Group the data by year and month, so that we see a time series of years and months
df_yr_mnth_group = meli_search_df.groupby(by=[meli_search_df.index.year, meli_search_df.index.month]).sum()
my_median = df_yr_mnth_group.median()
may_2020_total = float(df_yr_mnth_group.loc[(2020, 5)])
print(f'Median Google Search Traffic: {float(my_median)}')
print(f'Total for May, 2020: {may_2020_total}, an {round(float(((may_2020_total-my_median)/my_median)*100),2)}% increase in search traffic')

Median Google Search Traffic: 35201.0
Total for May, 2020: 38181.0, an 8.47% increase in search traffic


## The month of May, 2020 brought a meaningful increase in our search traffic above the median
We could use this data to aid our marketing team and our investor relations team coordinate SEO and other marketing efforts to drive search traffic towards our own press releases or paid sponsorship articles.

## Mine the Search Traffic Data for Seasonality
Next we want to see if we can *track* and *predict* interest in the company to help our marketing team concentrate their efforts around more optimal hours and days of the week. To do this we'll very cimple plot a heat map. Along the x-axis you'll see the hour of the day (military time), and alond the y-axis you'll see the day of week

In [176]:
# Plot a heat map with the hour as X and the Day of week as Y
meli_search_df.hvplot.heatmap(
    title = 'Daily HeatMap by Hour',
    x='index.hour',
    y='index.dayofweek',
    C= 'Search Trends',
    cmap= 'coolwarm'
).aggregate(function=np.mean)

:HeatMap   [index.hour,index.dayofweek]   (Search Trends)

### Peak Search Traffic Hours: 0:00-1:00 UTC , 22:00-23:00 UTC
### Peak Search Traffic Days of Week: Monday-Friday, with the best day being Tuesday

In [240]:
copy_df = meli_search_df.groupby(meli_search_df.index.hour).mean()
index_list = copy_df.index.to_list()
index_list = index_list[4:]
for x in range(1,4):
    index_list.append(x)
index_list
time_as_str = []
am_pm_str = ''
for ix in index_list:
    if ix < 16: 
        am_pm_str = 'am'
        if ix== 4:
            time_as_str.append(f'12:00{am_pm_str}')
        else:
            time = ix-4
            time_as_str.append(f'{time}:00{am_pm_str}')
    elif ix >= 16:
        am_pm_str = 'pm'
        if ix == 16:
            time_as_str.append(f'12:00{am_pm_str}')
        else:
            time = ix-16
            time_as_str.append(f'{time}:00{am_pm_str}')

time_as_str = time_as_str[:-3]
for x in range(8,12):
    time_as_str.append(f'{x}:00pm')
copy_df.index = time_as_str
copy_df


,Search Trends
12:00am,88.325974
1:00am,84.413255
2:00am,76.415205
3:00am,61.935023
4:00am,43.697206
5:00am,26.395062
6:00am,14.455491
7:00am,8.369071
8:00am,6.148148
9:00am,7.387914


In [241]:
copy_df.hvplot.bar(
    title= 'Hourly Search Traffic Average',
    rot = 90,
     color='Search Trends', cmap= 'bokeh')


:Bars   [index]   (Search Trends)

## Highest Search Volume times: 9:00 pm to 1:00 am EST
## Worst Search Volume times: 1:00 am to 7:00 am EST

In [272]:
big_season_df = meli_search_df.groupby(meli_search_df.index.weekofyear).mean()
median = float(big_season_df.median())
big_season_df['6 Week Avg'] = big_season_df.rolling(6).mean()
big_season_df= big_season_df.dropna()


C:\Users\Laramie\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  """Entry point for launching an IPython kernel.


In [274]:
fig = big_season_df.hvplot(title = 'Average Search Traffic by Week of Year')
hline = hv.HLine(median)
hline.opts(
    color = 'orange',
    line_dash= 'dashed',
    line_width= 2.0
)
fig * hline

:Overlay
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .HLine.I     :HLine   [x,y]

### From Week 40 onward, you see a steady increase in Average Search Trends

---

## Relating Search Traffic to Stock Price Patterns

Suppose the finance team wants to know if there was some coorelation among these search trends and share price -- we can represent graphically the relationship between Google search trends for MercadoLibre and the share price

In [282]:
meli_stock_df = pd.read_csv('resources/mercado_stock_price.csv')
meli_stock_df['date'] = pd.to_datetime(meli_stock_df['date'], infer_datetime_format=True)
meli_stock_df.head()

,date,close
0,2015-01-02 09:00:00,127.67
1,2015-01-02 10:00:00,125.44
2,2015-01-02 11:00:00,125.57
3,2015-01-02 12:00:00,125.40
4,2015-01-02 13:00:00,125.17


In [283]:
meli_search_df.head()

,Search Trends
Date,
2016-06-01 00:00:00+00:00,97
2016-06-01 01:00:00+00:00,92
2016-06-01 02:00:00+00:00,76
2016-06-01 03:00:00+00:00,60
2016-06-01 04:00:00+00:00,38
